In [35]:
import ray
import pandas as pd
import time 
import numpy as np
import binascii
import redis
import pprint
pp = pprint.PrettyPrinter() # for printing dicts and lists in a manner easy for the eyes



In [1]:
IDENTIFIER_LENGTH = 20

# This prefix must match the value defined in ray_redis_module.cc.

def hex_identifier(identifier):
    return binascii.hexlify(identifier).decode()


def identifier(hex_identifier):
    return binascii.unhexlify(hex_identifier)

def key_to_hex_identifiers(key):
    # Extract worker_id and task_id from key of the form
    # prefix:worker_id:task_id.
    offset = key.index(b":") + 1
    worker_id = hex_identifier(key[offset:(offset + IDENTIFIER_LENGTH)])
    offset += IDENTIFIER_LENGTH + 1
    task_id = hex_identifier(key[offset:(offset + IDENTIFIER_LENGTH)])
    return worker_id, task_id

import re
def clean(sometext):
    sometext = sometext.decode('UTF-8')
    ansi_escape = re.compile(r'\x1b[^m]*m')
    return ansi_escape.sub('', sometext)

In [3]:
ray.init(num_cpus=3)

Waiting for redis server at 127.0.0.1:24877 to respond...
Waiting for redis server at 127.0.0.1:28019 to respond...
Starting local scheduler with 3 CPUs and 0 GPUs.


{'local_scheduler_socket_names': ['/tmp/scheduler6480611'],
 'node_ip_address': '127.0.0.1',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store5536195', manager_name='/tmp/plasma_manager1209931', manager_port=63906)],
 'redis_address': '127.0.0.1:24877'}

In [4]:
# Function definitions...

@ray.remote
def example(x):
    time.sleep(np.random.random())
    return np.random.randn()

@ray.remote
class TestCls():
    def __init__(self):
        self.g = 1
        
    def to_go(self, x):
        return x

    
@ray.remote
class Outer():
    def __init__(self):
        self.f = 1
        self.test = TestCls.remote()
    
    def to_go2(self, x):
        return x * 2
    
    def error(self):
        return 1/0

In [5]:
# Here, we generate data in redis for remote tasks
results = ray.get([example.remote(x) for x in range(4)])

# Generating data for Actor tasks
actor = TestCls.remote()
actor_results = ray.get([actor.to_go.remote(1)])

In [6]:
global_state = ray.worker.global_state # Used to access redis client, but also has nice interface for certain information 
gworker = ray.worker.global_worker # Unused for now
rc_non = global_state.redis_client # Redis Client for interacting with redis without decoding
addr, port = gworker.redis_address.split(":")
rc = redis.StrictRedis(host=addr, port=port, decode_responses=True)

Key things included in redis from `rc.keys()`:
 - Event log
 - worker info
 - Remote functions
 - "Functions to run"
 - Drivers
 - Redis clients
 - Actor classes
 - Actors

In [ ]:

for k in rc.keys():
    try:
        print(k)
        pp.pprint(rc.hgetall(k))
#         rc.hgetall(k) 
    except Exception:
        print(k, "Failed ")
# Certain return values, such as `event_log:*`, can only be accessed via list calls to redis

#         try:
#             print('#' * 10)
#             print(k)
#             pp.pprint(rc.lrange(k, 0, -1))
#         except Exception:
#             print(k, "Failed ")
            

## Actor Info

NOTE: 
# TODO: Fix actor_id

In [42]:
# import unicode 
actor_list = []
for a_key in rc.keys("Actor:*"):
    v = rc.hgetall(a_key)
    v['actor_id'] = a_key
    v['class_id'] = hex_identifier(v['class_id'])
    actor_list.append(v)

actor_df = pd.DataFrame(actor_list)

In [43]:
actor_df

,actor_id,class_id,num_gpus
0,"Actor:b'[\xdfYzv\x0bdG#,P\xd1&,\xed\xb6\x91\xf...",abc1797efc03353d22511cf58cca68bbdc19a8bf,0


In [44]:
actor_classes = []
for a_key in rc.keys("ActorClass:*"):
    
    v = rc.hgetall(a_key)
    del v['class'] # removed pickle hex for readability
    v['driver_id'] = hex_identifier(v['driver_id'])
    class_id = a_key.split(b':')[1]
    v['class_id'] = hex_identifier(class_id)
    actor_classes.append(v)
    
actor_class_df = pd.DataFrame(actor_classes)

In [45]:
actor_class_df

,actor_method_names,class_id,class_name,driver_id,module
0,"[""to_go"", ""__init__"", ""__ray_terminate__""]",abc1797efc03353d22511cf58cca68bbdc19a8bf,Class,aba3b41b39cdaebac7d7e77de0fb139fa6ecaedf,ray.actor


## Function Info

In [46]:
fn_table = global_state.function_table()
fn_list = []
for fn_id in fn_table:
    val = fn_table[fn_id]
    val["function_id"] = fn_id
    fn_list.append(val)
pd.DataFrame(fn_list)

,DriverID,Module,Name,function_id
0,aba3b41b39cdaebac7d7e77de0fb139fa6ecaedf,__main__,__main__.example,cf1fc498b8a2b4cb79bc2ed5abf074ba96ae69a1


## Node Info

In [47]:
ctable = global_state.client_table()

client_list = []
for node_ip in ctable:
    for client in ctable[node_ip]:
        client["node_ip_address"] = node_ip
        client_list.append(client)

client_df = pd.DataFrame(client_list)
client_df

,AuxAddress,ClientType,DBClientID,Deleted,LocalSchedulerSocketName,NumCPUs,NumGPUs,node_ip_address
0,NaN,global_scheduler,432c37b704ca8c75ab464bbf39333a75ee267af1,False,NaN,NaN,NaN,127.0.0.1
1,127.0.0.1:63906,local_scheduler,2998397411b0c5cf0d8f764f3c510838ae4cc709,False,/tmp/scheduler6480611,3.0,0.0,127.0.0.1
2,NaN,plasma_manager,6ebdbcc87af95df809bb294ac3c8768f9a31231e,False,NaN,NaN,NaN,127.0.0.1


In [54]:
object_dict = {oid.hex(): v for oid, v in global_state.object_table().items()}
object_df = pd.DataFrame(object_dict).transpose()
object_df
# May need to change this so ID is not index

,IsPut,ManagerIDs,TaskID
5d14f428a07abb69994fcfea135e04a847769264,False,[6ebdbcc87af95df809bb294ac3c8768f9a31231e],5c14f428a07abb69994fcfea135e04a847769264
aca487e1c1d2b301d101b26d04b361b96447cb62,False,[6ebdbcc87af95df809bb294ac3c8768f9a31231e],ada487e1c1d2b301d101b26d04b361b96447cb62
baaf4525690b4daef75f9e227c610d89dfe8c681,False,[6ebdbcc87af95df809bb294ac3c8768f9a31231e],bbaf4525690b4daef75f9e227c610d89dfe8c681
c4a2bc73bbba18c7a5683f59b9e6701970bb867c,False,[6ebdbcc87af95df809bb294ac3c8768f9a31231e],c5a2bc73bbba18c7a5683f59b9e6701970bb867c
e1108437b79bd7ea0ba07ed6b98cd8bd71043c7b,False,[6ebdbcc87af95df809bb294ac3c8768f9a31231e],e0108437b79bd7ea0ba07ed6b98cd8bd71043c7b
f0213b56d9a8c72fde8e437e2d57b7a281156072,False,[6ebdbcc87af95df809bb294ac3c8768f9a31231e],f1213b56d9a8c72fde8e437e2d57b7a281156072


# Tasks

In [55]:
tt = global_state.task_table()
tt_list = list(tt.values())

for d in tt_list:
    d['TaskSpec']['ReturnObjectIDs'] = [oid.hex() for oid in d['TaskSpec']['ReturnObjectIDs']]

In [56]:
from pandas.io.json import json_normalize

result = json_normalize(tt_list)

result

,LocalSchedulerID,State,TaskSpec.ActorCounter,TaskSpec.ActorID,TaskSpec.Args,TaskSpec.DriverID,TaskSpec.FunctionID,TaskSpec.ParentCounter,TaskSpec.ParentTaskID,TaskSpec.RequiredResources.CPUs,TaskSpec.RequiredResources.GPUs,TaskSpec.ReturnObjectIDs,TaskSpec.TaskID
0,2998397411b0c5cf0d8f764f3c510838ae4cc709,16,2,ffffffffffffffffffffffffffffffffffffffff,[2],aba3b41b39cdaebac7d7e77de0fb139fa6ecaedf,cf1fc498b8a2b4cb79bc2ed5abf074ba96ae69a1,2,d38c8ab355c8c3469db5390590975b936145af53,1.0,0.0,[aca487e1c1d2b301d101b26d04b361b96447cb62],ada487e1c1d2b301d101b26d04b361b96447cb62
1,ffffffffffffffffffffffffffffffffffffffff,8,0,ffffffffffffffffffffffffffffffffffffffff,[],aba3b41b39cdaebac7d7e77de0fb139fa6ecaedf,ffffffffffffffffffffffffffffffffffffffff,0,099e877ab3fbb29235c6971b4df3f64a2ac3c6fd,0.0,0.0,[],d38c8ab355c8c3469db5390590975b936145af53
2,2998397411b0c5cf0d8f764f3c510838ae4cc709,16,1,5bdf597a760b6447232c50d1262cedb691fe2f45,[1],aba3b41b39cdaebac7d7e77de0fb139fa6ecaedf,7916fc26bb3c69a23ba4e6714b02a2c07e8542f2,5,d38c8ab355c8c3469db5390590975b936145af53,1.0,0.0,[5d14f428a07abb69994fcfea135e04a847769264],5c14f428a07abb69994fcfea135e04a847769264
3,2998397411b0c5cf0d8f764f3c510838ae4cc709,16,0,ffffffffffffffffffffffffffffffffffffffff,[0],aba3b41b39cdaebac7d7e77de0fb139fa6ecaedf,cf1fc498b8a2b4cb79bc2ed5abf074ba96ae69a1,0,d38c8ab355c8c3469db5390590975b936145af53,1.0,0.0,[e1108437b79bd7ea0ba07ed6b98cd8bd71043c7b],e0108437b79bd7ea0ba07ed6b98cd8bd71043c7b
4,2998397411b0c5cf0d8f764f3c510838ae4cc709,16,0,5bdf597a760b6447232c50d1262cedb691fe2f45,[],aba3b41b39cdaebac7d7e77de0fb139fa6ecaedf,1939f247b3ae06244a8d50fa7dabbcc975c6fc4c,4,d38c8ab355c8c3469db5390590975b936145af53,1.0,0.0,[c4a2bc73bbba18c7a5683f59b9e6701970bb867c],c5a2bc73bbba18c7a5683f59b9e6701970bb867c
5,2998397411b0c5cf0d8f764f3c510838ae4cc709,16,1,ffffffffffffffffffffffffffffffffffffffff,[1],aba3b41b39cdaebac7d7e77de0fb139fa6ecaedf,cf1fc498b8a2b4cb79bc2ed5abf074ba96ae69a1,1,d38c8ab355c8c3469db5390590975b936145af53,1.0,0.0,[f0213b56d9a8c72fde8e437e2d57b7a281156072],f1213b56d9a8c72fde8e437e2d57b7a281156072
6,2998397411b0c5cf0d8f764f3c510838ae4cc709,16,3,ffffffffffffffffffffffffffffffffffffffff,[3],aba3b41b39cdaebac7d7e77de0fb139fa6ecaedf,cf1fc498b8a2b4cb79bc2ed5abf074ba96ae69a1,3,d38c8ab355c8c3469db5390590975b936145af53,1.0,0.0,[baaf4525690b4daef75f9e227c610d89dfe8c681],bbaf4525690b4daef75f9e227c610d89dfe8c681


We can also parse the event logs in order to get fine grained timing for remote tasks. However, as a user, I'd probably only care about time taken in running the task -- this can be much refined.

In [57]:
import json 
event_list = []

# Get and decode all task timing/event logs
for key in rc.keys("event_log*"):
    content = rc.lrange(key, 0, -1)
#     event_list.append(json.loads(content[0].decode())) 
    event_list.append(json.loads(content[0]))
    
# event_list

In [58]:
from collections import defaultdict

# This seems to be the best way to do the event_log -> dataframe pipeline. 
# First generate a (key, [value]) mapping for all tasks and then apply some pandas operations to convert.

# event_dict is used to store timing info
event_dict = defaultdict(lambda: np.full(len(event_list), np.nan))

# info_dict is used to store meta data - such as function names and task id
info_dict = defaultdict(lambda: [None] * len(event_list))

for i, task_event in enumerate(event_list):
    for event in (task_event):
        time, label, startstop, info = event
        event_dict[(label, startstop)][i] = time
        if info:
            for k in info:
                info_dict[k][i] = info[k]

In [59]:
# The tuple keys for `event_dict` produce a hierarchical index, which could be useful. However, joining or merging it 
# with other non-hierarchical indices will throw away this structure.

edf = pd.DataFrame(event_dict) 
edf.rename(columns={1: 'start', 2:'end'}, inplace=True)
edf

ray:acquire_lock                ray:get_task                \
             start           end         start           end   
0     1.496825e+09  1.496825e+09  1.496825e+09  1.496825e+09   
1     1.496825e+09  1.496825e+09  1.496825e+09  1.496825e+09   
2     1.496825e+09  1.496825e+09  1.496825e+09  1.496825e+09   
3     1.496825e+09  1.496825e+09  1.496825e+09  1.496825e+09   
4     1.496825e+09  1.496825e+09  1.496825e+09  1.496825e+09   
5     1.496825e+09  1.496825e+09  1.496825e+09  1.496825e+09   

  ray:import_remote_function                    ray:task                \
                       start           end         start           end   
0               1.496825e+09  1.496825e+09  1.496825e+09  1.496825e+09   
1               1.496825e+09  1.496825e+09  1.496825e+09  1.496825e+09   
2                        NaN           NaN  1.496825e+09  1.496825e+09   
3               1.496825e+09  1.496825e+09  1.496825e+09  1.496825e+09   
4                        NaN           NaN  1.496825e+09  1.496825e+09   
5                        NaN           NaN  1.496825e+09  1.496825e+09   

  ray:task:execute               ray:task:get_arguments                \
             start           end                  start           end   
0     1.496825e+09  1.496825e+09           1.496825e+09  1.496825e+09   
1     1.496825e+09  1.496825e+09           1.496825e+09  1.496825e+09   
2     1.496825e+09  1.496825e+09           1.496825e+09  1.496825e+09   
3     1.496825e+09  1.496825e+09           1.496825e+09  1.496825e+09   
4     1.496825e+09  1.496825e+09           1.496825e+09  1.496825e+09   
5     1.496825e+09  1.496825e+09           1.496825e+09  1.496825e+09   

  ray:task:store_outputs               ray:wait_for_function                
                   start           end                 start           end  
0           1.496825e+09  1.496825e+09          1.496825e+09  1.496825e+09  
1           1.496825e+09  1.496825e+09          1.496825e+09  1.496825e+09  
2           1.496825e+09  1.496825e+09          1.496825e+09  1.496825e+09  
3           1.496825e+09  1.496825e+09          1.496825e+09  1.496825e+09  
4           1.496825e+09  1.496825e+09          1.496825e+09  1.496825e+09  
5           1.496825e+09  1.496825e+09          1.496825e+09  1.496825e+09

In [60]:
idf = pd.DataFrame(info_dict)
# idf.columns = pd.MultiIndex.from_tuples([(c, '') for c in idf]) # this is non-idempotent!
idf

,function_name,task_id
0,__main__.example,f1213b56d9a8c72fde8e437e2d57b7a281156072
1,__main__.example,ada487e1c1d2b301d101b26d04b361b96447cb62
2,__main__.example,bbaf4525690b4daef75f9e227c610d89dfe8c681
3,__main__.example,e0108437b79bd7ea0ba07ed6b98cd8bd71043c7b
4,__init__,c5a2bc73bbba18c7a5683f59b9e6701970bb867c
5,to_go,5c14f428a07abb69994fcfea135e04a847769264


In [61]:
remote_table = pd.concat([idf, edf], axis=1)

In [ ]:
# Here, we join two tables together as an example
idf.merge(result, left_on="task_id", right_on="TaskSpec.TaskID")

In [ ]:
# Example to get Error, function_name, IP address, Actor ID, ParentID 

# TODO:

- Error messages logging (specifics - which node, which function call, which actor, what time)
  - Getting Error messages from Redis is redundant because we already get info in the event_log. Error messages provide extra information such as `error_id` and `type`, which don't seem particularly useful.
  - ~Create table for workers (Id, socket info, node IP address) ... are workers even still a proper abstraction~
  - Get multinode setting - test out client table
  - !! Write out an example for error tracing